# Lesson 2. Introduction to the `sf` package

In this lesson we'll learn about `sf`, the package that is core to using geospatial data in R. We'll go through the structure of the data (it's not too different from regular data.frames!), geometries, shapefiles, and how to save your hard work.

- 2.1 What is the `sf` package?
- 2.2 Read in a shapefile
- 2.3 Explore the `sf` object
- 2.4 Plot the `sf` object
- 2.5 Subset the `sf` object
- 2.6 Save your data
- 2.7 Recap
- **Exercise**: IO, Manipulation, and Mapping

<br>
<font color='grey'>
    <b>Instructor Notes</b>

- Datasets used
    - 'notebook_data/california_counties/CaliforniaCounties.shp'
    - 'notebook_data/census/Places/cb_2018_06_place_500k.zip'

- Expected time to complete
    - Lecture + Questions: 30 minutes
    - Exercises: 5 minutes
</font>

## 2.1 What is `sf`?

### `sf` and related geospatial packages

[`sf`](https://r-spatial.github.io/sf/) is a relatively new package that makes it easier to work with geospatial data in R. In the last few years it has grown more powerful and stable. This is really great because the previous means of working with geospatial data in R (the `sp` package) was more complicated. `sf` is now the go-to package for working with `vector` geospatial data in R. 

> **Protip**: If you work with `raster` data you will want to checkout the [raster](https://cran.r-project.org/web/packages/raster/index.html) package. We will not cover raster data in this tutorial.

### `sf` = <u>s</u>imple <u>f</u>eatures
`sf` creates geospatial `data.frame`s that retain all of the functionality of R `data.frame`s, but which are extended with a geometry column and with geospatial metadata, making it easy to process your data using both standard table-based operations and explicitly geospatial operations.



### Load `sf`

Let's start by loading the `sf` library.


In [ ]:
library(sf)



## 2.2 Read in a shapefile

As we discussed in the initial geospatial overview, a *shapefile* is one type of geospatial data that holds vector data. 

> To learn more about ESRI Shapefiles, this is a good place to start: [ESRI Shapefile Wiki Page](https://en.wikipedia.org/wiki/Shapefile) 

The tricky thing to remember about shapefiles is that they're actually a collection of 3 to 9+ files together. Here's a list of all the files that can make up a shapefile:
 
>`shp`: The main file that stores the feature geometry
>
>`shx`: The index file that stores the index of the feature geometry  
>
>`dbf`: The dBASE table that stores the attribute information of features 
>
>`prj`: The file that stores the coordinate system information. (should be required!)
>
>`xml`: Metadata —Stores information about the shapefile.
>
>`cpg`: Specifies the code page for identifying the character set to be used.

But it remains the most commonly used file format for vector spatial data, and it's really easy to visualize in one go!

Let's try it out with California counties, and use `sf` for the first time. `sf::st_read` is a flexible function that let's you read in many different types of geospatial data.


In [ ]:
# Read in the counties shapefile
counties = st_read('notebook_data/california_counties/CaliforniaCounties.shp')


In [ ]:
# Plot out California counties
plot(counties)


Wow! That gives us a plot grid of up to the first 9 attributes (i.e. columns) in our dataset.

And what if we just want to plot a single variable?


In [ ]:
plot(counties['MED_AGE_M'])



Wow! So easy! We just made a **choropleth** map of median male age, by county!

We're off to a running start.

## 2.3 Explore the `sf data.frame`

Before we get in too deep, let's discuss what a `sf data.frame` is, and how it's different from a plain `data.frame`.

### The `sf data.frame`

An `sf data.frame`, a.k.a. an `sf` object, is just like a plain `data.frame`, but with an extra geometry column. The `sf` package then has a variety of geospatial functions that use that geometry column. 

I repeat because it's important:

> `An sf object is a plain data.frame with a geometry column added.`

> This means all the normal operations that we can run on `data.frame`s will also work on `sf` objects!

With that in mind, let's start exploring our `sf` object just like we would a dataset in a plain `data.frame`.


In [ ]:
# Find the number of rows and columnds in counties
dim(counties)


In [ ]:
# Look at the first couple of rows in our sf object
head(counties)


In [ ]:
# Look at all the variables included in our data
colnames(counties)


It looks like we have a good amount of information about the total population for different years and the densities, as well as race, age, and occupancy info.

## 2.4 Plotting `sf` objects

We're able to map our `sf` object because of the extra `geometry` column.

### `sf` Geometries
There are three main types of geometries that can be associated with your `sf` object: points, lines and polygons:

<img src ="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-vector/pnt_line_poly.png" width="450"></img>

In an `sf data.frame` these geometries are encoded in a format known as [Well-Known Text (WKT)](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry). For example:

> - POINT (30 10)
> - LINESTRING (30 10, 10 30, 40 40)
> - POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))
>
> *where coordinates are separated by a space and coordinate pairs by a comma*

Your `sf` object may also include the variants **multipoints, multilines, and multipolgyons** if some of the features are composed multiple parts. For example, if we had data representing US states (one per row), we could use a POLYGON geometry for states like Utah or Colorado, but would need a MULTIPOLYGON for states like Hawaii, which includes many islands.


<img src="http://www.pngall.com/wp-content/uploads/2016/03/Light-Bulb-Free-PNG-Image.png" width="20" align=left >  **Question** What kind of geometry would a roads `sf` object have? What about one that includes landmarks in the San Francisco Bay Area?




Just like with other plots we can make in R, we can start customizing our maps' colors, title, etc.

The most rudimentary way to do this would be to use base R's `plot` function to plot our `sf` objects geometries.


In [ ]:
# Plot our geometries, coloring them pale, with dark green borders
plot(counties$geometry, col='tan', border='darkgreen', main="CA counties")


However, we'll get much more customizability if we use a special-purpose mapping package, rather than just relying on `sf` methods of base R functions.

Our go-to mapping package of choice will be [`tmap`](https://github.com/mtennekes/tmap). Its name stands for "thematic maps", i.e. maps in which we use dimensions of our dataset to control the visualization parameters of our maps, thus creating effective data visualizations. 

You'll get plenty of introduction here in the workshop, but for additional support you can check out the [tmap vignette](https://cran.r-project.org/web/packages/tmap/vignettes/tmap-getstarted.html) or Google other tutorials and references.

Let's start by loading the package and creating a 'quick tmap'.


In [ ]:
# load tmap
library(tmap)

# plot a 'quick tmap'
qtm(counties)


Nice!

That's the quickest, simplest example of a static map that `tmap` can make. However, `tmap` has 2 modes:
    - 'plot' mode: static maps
    - 'view' mode: interactive maps

`tmap` loads up in 'plot' mode. Let's switch it to 'view' mode and then take a look at that same map. 

We could either set the mode explicitly (`tmap_mode('view')`) or just toggle back and forth using the `ttm` ('toggle tmap mode') function. Let's use the latter.


In [ ]:
# toggle the mode
ttm()
# then make our quick tmap again
qtm(counties)


That's outstanding! We get a clickable, scrollable, zoomable map built in Javascript's Leaflet package... right out of the box!

And to create thematic maps, we can use `tmap`'s more verbose mapping functions to create a new `tmap` object and then add geometry layers to it, setting different aesthetic aspects of those layers.

For now, let's recreate that same map from above, but this time using `tmap` instead of the `sf` method of base R's `plot`.


In [ ]:
tm_shape(counties) +  # use the `tm_shape` function to create a tmap object
  tm_polygons(col='tan', border.col='darkgreen', # add `tm_polygons` layer, coloring as before, 
              alpha=0.5) # & making transparent


Nice! Looks pretty much the same as above, except now it's interactive, and overlaid on a super-sweet basemap!

Now we have two mapping methods:
    - base R `plot`: nice for simple, pared down plotting tasks
    - `tmap`: quick maps, both static and interactive, with greater flexibility

## 2.5 Subset the `sf` object

Since we'll be focusing on Berkeley later in the workshop, let's subset our `sf` object to just be for Alameda County.


In [ ]:
# See the vector of all county names included in our dataset
counties$'NAME'


It looks like Alameda county is specified as "Alameda" in this dataset.



In [ ]:
counties[counties$NAME == 'Alameda',]



Now we can create a new `sf` object called `alameda_county` that is a subset of our counties geodataframe.



In [ ]:
alameda_county = counties[counties$NAME == 'Alameda',]



In [ ]:
# Plot our newly subsetted sf object
plot(alameda_county$geometry, col='pink', border='green', lwd=5, main='Why not?')


Nice! Looks like we have what we were looking for.

*FYI*: You can also make dynamic plots of one or more county without saving to a new gdf.


In [ ]:
bay_area_counties = c('Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco', 
                      'San Mateo', 'Santa Clara', 'Santa Cruz', 'Solano', 'Sonoma')
qtm(counties[counties$NAME %in% bay_area_counties,])


## 2.6 Save your Data

Let's not forget to save out our `alameda_county` object. This way we won't need to repeat the processing steps and attribute join we did above.

We can save it as a shapefile.


In [ ]:
st_write(alameda_county, "outdata/alameda_county.shp", delete_dsn=T)



One of the problems of saving to a shapefile is that our column names get truncated to 10 characters (a shapefile limitation.) 

Instead of renaming all columns with obscure names that are less than 10 characters, we can save our `sf` object to a spatial data file format that does not have this limation - [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) or [GPKG](https://en.wikipedia.org/wiki/GeoPackage) (geopackage) file.
- These formats have the added benefit of outputting only one file in contrast tothe multi-file shapefile format.


In [ ]:
st_write(alameda_county, "outdata/alameda_county.json", driver="GeoJSON", delete_dsn=T)



In [ ]:
st_write(alameda_county, "outdata/alameda_county.gpkg", driver="GPKG", delete_dsn=TRUE)



You can read these in, just as you would a shapefile, with `st_read`



In [ ]:
gpkg_test = st_read("outdata/alameda_county.gpkg")
plot(gpkg_test)


In [ ]:
json_test = st_read("outdata/alameda_county.json")
plot(json_test)


There are also many other formats we could use for data output.

**NOTE**: If you're working with point data (i.e. a single latitude and longitude value per feature),
then CSV might be a good option!

## 2.7 Recap

In this lesson we learned about...

- The `sf` package 
- Reading in shapefiles 
    - `st_read`
- `sf` data structures
    - `dim`, `head`, `colnames`, `str`
- Plotting `sf` objects
    - `plot`
    - `tmap`
- Subsetting `sf` objects
    - matrix subsetting syntax
- Saving `sf` objects to file
    - `st_write`

## Exercise: IO, Manipulation, and Mapping

Now you'll get a chance to practice the operations we learned above.

In the following cell, compose code to:

1. Read in the California places data (`notebook_data/census/Places/cb_2018_06_place_500k.shp`)
2. Subset the data to Berkeley
3. Plot using base R `plot`, and customize as desired
4. Save out as a shapefile (`outdata/berkeley_places.shp`)



To see the solution, look at the hidden text below.


In [ ]:
# YOUR CODE HERE



## Solution hidden here!

<!--
# SOLUTION

# 1. Read in the California places data 
california_places = st_read('./notebook_data/census/Places/cb_2018_06_place_500k.shp')
head(california_places)

# 2. Subset the data to Berkeley
berkeley_places = california_places[california_places$NAME=='Berkeley',]

# 3. Plot and customize
plot(berkeley_places, col='lightgreen', border='grey')

# 4. Save to a shapefile
st_write(berkeley_places, "outdata/berkeley_places.shp")

-->

---
<div style="display:inline-block;vertical-align:middle;">
<a href="https://dlab.berkeley.edu/" target="_blank"><img src ="assets/images/dlab_logo.png" width="75" align="left">
</a>
</div>

<div style="display:inline-block;vertical-align:middle;">
<div style="font-size:larger">&nbsp;D-Lab @ University of California - Berkeley</div>
<div>&nbsp;Team Geo<div>
</div>
        
